In [1]:
import pandas as pd

### Задание 1

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [94]:
# нашли значения больше 100
above_100_prep = ratings.groupby(['userId']).count().reset_index()[['userId','movieId']]
user_id = pd.DataFrame(above_100_prep[(above_100_prep['movieId']>=100)]['userId'])
user_id.head()

,userId
3,4
4,5
7,8
14,15
16,17


In [95]:
# нашли максимальное и минимальное значение
max_values = ratings.groupby(['userId']).max().reset_index()[['userId','timestamp']]
min_values = ratings.groupby(['userId']).min().reset_index()[['userId','timestamp']]
max_values.head()

,userId,timestamp
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145


In [100]:
# добавили макимальное и минимальное значение timestamp к userId

user_id_w_max = user_id.merge(max_values, how = 'left', on='userId' )
user_id_w_max.columns = ['userId','timestamp_max']

user_id_w_max_min = user_id_w_max.merge(min_values, how = 'left', on='userId' )
user_id_w_max_min.columns = ['userId','timestamp_max','timestamp_min']

user_id_w_max_min.head()

,userId,timestamp_max,timestamp_min
0,4,949982274,949778714
1,5,1163375145,1163373044
2,8,1154474527,1154389340
3,15,1469330735,997937239
4,17,1127476640,1127468587


In [101]:
# функция расчета разницы между максимальным и минимальным значением timestamp
def diff_func (data):
    return data.timestamp_max - data.timestamp_min


In [104]:
user_id_w_max_min['diff']= user_id_w_max_min.apply(diff_func,axis=1)
user_id_w_max_min.head()

,userId,timestamp_max,timestamp_min,diff
0,4,949982274,949778714,203560
1,5,1163375145,1163373044,2101
2,8,1154474527,1154389340,85187
3,15,1469330735,997937239,471393496
4,17,1127476640,1127468587,8053


In [119]:
print(f"Среднее время жизни пользователей в днях составляет {round(user_id_w_max_min.mean()['diff']/86400,3)}")


Среднее время жизни пользователей в днях составляет 455.229


### Задание 2

In [129]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [122]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [123]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [124]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [127]:
df_cients = pd.DataFrame(client_base.client_id)
df_cients

,client_id
0,111
1,112
2,113
3,114
4,115
5,116
6,117
7,118


In [136]:
df_cients = pd.DataFrame(client_base.client_id)
df_cients_w_rzd = df_cients.merge(rzd, how = 'left', on='client_id')
df_cients_w_rzd_auto = df_cients_w_rzd.merge(auto, how = 'left', on='client_id')
df_cients_w_rzd_auto_air = df_cients_w_rzd_auto.merge(air, how = 'left', on='client_id')
final_revenue_table = df_cients_w_rzd_auto_air.fillna(0)
final_revenue_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [138]:
final_revenue_table_w_adrres = final_revenue_table.merge(client_base, how = 'left', on='client_id')
final_revenue_table_w_adrres

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
